In [135]:
import json
import pandas as pd
import os
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import List
from pydantic import BaseModel, Field
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
model = "gpt-4o-2024-08-06"

In [136]:
# Get human emotions

# Define the Pydantic model for the API response
class EmotionsResponse(BaseModel):
    Characteristics: List[str] = Field(None, description="List of non-redundant human emotions.")

def get_emotions(model: str) -> List[str]:
    """Gets a list of 50 non-redundant human emotions using the specified gpt model."""
    
    # Define system and user prompts
    system_prompt = "Find 50 non-redundant and different human emotions. "\
    "For example, from each of these four emotion groups, only select one representative"\
    ": 1: joy, happiness, 2: Shame, Embarrassment, "\
    "3: Envy, Jealousy , 4: Hate, disgust, hatered, Resentment. "\
    "So on and so forth."

    user_prompt = "Select 50 non-redundant and different human emotions."

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": system_prompt},
                {"role": "system", "content": "Did you accidentally select multiple emotions from the four emotion groups? If so, keep only one representative from each group and drop the rest."},
                {"role": "system", "content": "Check again for redundancy. Remember, you must only select emotions describing non-redundant human feelings."},
                {"role": "user", "content": user_prompt}
            ],
            response_format=EmotionsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return []

# Example usage
emotions = get_emotions(model= model)

In [ ]:
# #to check emotion redundancy by looking at example groups 
#[i for i in list(json.loads(emotions).values())[0] if i in ['Joy', 'Happiness', 'Shame', 'Embarrassment', 'Envy', 'Jealousy' , 'Hate', 'disgust', 'hatered', 'Resentment']]

In [137]:
#Get 100 best selling American clothing brands 

# Define the Pydantic model for the API response
class BrandsResponse(BaseModel):
    Brands: List[str] = Field(None, description="Brands as a list of strings.")

def get_brands(model: str) -> List[str]:
    """Get 100 best selling American clothing brands using the specified gpt model."""

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": "Find 100 non-redundant best selling American clothing brands."},
                {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
                {"role": "user", "content": "Give me 100 best selling American clothing brands."}
            ],
            response_format=BrandsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return []

# Example usage
brands = get_brands(model= model)

In [138]:
emotions_ls = list(json.loads(emotions).values())[0]
brands_ls = list(json.loads(brands).values())[0]

In [139]:
# Embedding brand in emotions space: Get association scores between an input and list of emotions

Characteristic = Enum('Characteristic', dict([(emotion, emotion) for emotion in emotions_ls]))

class EmotionalAssociationScore(BaseModel):
    emotion: Characteristic
    score: float

class EmotionalAssociationScores(BaseModel):
    associations: List[EmotionalAssociationScore] = Field(description="A list of emotions and associated scores")

def emotional_association_scores(
        thing, 
        model,
        emotions
    ):
    
    prompt = f"Assign emotional association scores between {0} and {len(emotions)} for the provided thing. "\
    "Assign a score for each of the following emotions. Briefly, explain the reason behind the association score."\
    "Ensure the scores reflect the association strength for the specified thing. "\
    "Thing: "\
    f"{thing}"
            
    completion = client.beta.chat.completions.parse(
        model = model,
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format=EmotionalAssociationScores,
    )
    #output returns in the defined pydantic style
    output = completion.choices[0].message.parsed
    return thing, output.json()

In [137]:
#not using this for the moment
# #Embedding brands in emotions space: 
# # tried nested prompt but decided to go with one prompt and a list comprehension
# emotions= emotions_ls
# associations_brands = [emotional_association_scores(thing, model, emotions) for thing in brands_ls[:3]]


[('Nike',
  '{"associations": [{"emotion": "Joy", "score": 42.0}, {"emotion": "Shame", "score": 15.0}, {"emotion": "Envy", "score": 35.0}, {"emotion": "Hate", "score": 20.0}, {"emotion": "Surprise", "score": 30.0}, {"emotion": "Anxiety", "score": 10.0}, {"emotion": "Awe", "score": 28.0}, {"emotion": "Curiosity", "score": 25.0}, {"emotion": "Melancholy", "score": 5.0}, {"emotion": "Contentment", "score": 40.0}, {"emotion": "Gratitude", "score": 38.0}, {"emotion": "Fear", "score": 12.0}, {"emotion": "Loneliness", "score": 7.0}, {"emotion": "Nostalgia", "score": 18.0}, {"emotion": "Excitement", "score": 44.0}, {"emotion": "Boredom", "score": 5.0}, {"emotion": "Romantic", "score": 2.0}, {"emotion": "Wonder", "score": 32.0}, {"emotion": "Empathy", "score": 22.0}, {"emotion": "Frustration", "score": 15.0}, {"emotion": "Anticipation", "score": 36.0}, {"emotion": "Confidence", "score": 48.0}, {"emotion": "Despair", "score": 8.0}, {"emotion": "Sympathy", "score": 20.0}, {"emotion": "Longing", "

In [141]:
def get_df(thing, model, emotions):
    gpt = emotional_association_scores(thing, model, emotions)
    data = list(json.loads(gpt[1]).values())[0]
    df = pd.DataFrame(data)
    df.rename(columns = {'score': gpt[0]}, inplace=True)
    df.set_index('emotion', inplace=True)
    return df

def get_dfs(things_ls, model, emotions):
    merged_df = pd.DataFrame()
    for thing in things_ls[:2]:
        new_df = get_df(thing, model, emotions)
        if merged_df.empty:
            merged_df = new_df
        else:
            merged_df = pd.merge(merged_df, new_df, left_index=True, right_index=True, how='outer')
    return merged_df


things_ls = brands_ls
dfs = get_dfs(things_ls, model, emotions)
dfs

,Nike,Ralph Lauren
emotion,,
Joy,512.0,500.0
Shame,72.0,100.0
Envy,389.0,350.0
Disgust,55.0,50.0
Fear,120.0,50.0
Surprise,230.0,200.0
Sadness,48.0,100.0
Anger,110.0,50.0
Anticipation,475.0,400.0


In [142]:
#Embedding a book in emotions space: 
emotions= emotions_ls
thing = 'Summer Island'
df = get_df(thing, model, emotions)
df


,Summer Island
emotion,
Joy,43.0
Shame,5.0
Envy,28.0
Disgust,7.0
Fear,11.0
Surprise,21.0
Sadness,13.0
Anger,8.0
Anticipation,39.0


In [63]:
d = list(json.loads(associations_book).values())[0]
columns = [k for k,v in d[0].items()]
df  = pd.DataFrame.from_dict(d)

['emotion', 'score']

In [ ]:
df = pd.DataFrame.from_dict(json.loads(associations_book))

df['emotions'] = df['associations'].apply(lambda x: list(x.values())[0])
df['book_score'] = df['associations'].apply(lambda x: list(x.values())[1])
df.drop(columns=['associations'], inplace=True)
df.set_index('emotions', inplace=True)
df



In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

# Define your vectors
A = np.array([[2, 3]])
B = np.array([[5, 4]])

# L2 normalize the vectors
A_normalized = normalize(A, norm='l2')
B_normalized = normalize(B, norm='l2')

# Calculate cosine similarity on normalized vectors
similarity = cosine_similarity(A_normalized, B_normalized)

print(similarity)  # Output will reflect the cosine similarity of the normalized vectors


In [40]:
#The cosine similarity ranges from -1 to 1, where:
#1 indicates identical vectors (i.e., vectors point in the same direction).
#0 indicates orthogonality (i.e., vectors are at a 90-degree angle to each other, no similarity).
#-1 indicates opposite directions (i.e., vectors point in exactly opposite directions).
#represents similarity between feature vectors, quantifying similarity between two vectors based on their direction, 
# irrespective of their magnitude.





[[0.95292578]]


In [33]:

from sklearn.metrics.pairwise import cosine_similarity
# Example Pandas Series
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6])

# Reshape Series to 2D array (required by cosine_similarity)
s1_reshaped = s1.values.reshape(1, -1)
s2_reshaped = s2.values.reshape(1, -1)

# Cosine Similarity Calculation
cosine_sim = cosine_similarity(s1_reshaped, s2_reshaped)
cosine_sim
#print(f'Cosine Similarity: {cosine_sim[0][0]}')

(3,)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = []
for adj_A in list(.values()):
    avg_embedding_A = average_embedding(get_embeddings(adj_A))
    for adj_B in list(d_s.values()):
        avg_embedding_B = average_embedding(get_embeddings(adj_B))
        similarity_score = cosine_similarity([avg_embedding_A], [avg_embedding_B])[0][0]
        similarities.append((adj_A, adj_B, similarity_score))

In [ ]:
# a method
#embedding dimension is emotions
#talk about options
#get the brands, go through 50 emotins at a time
#cosine: normalize first: l2 norm = 1
#give instructions on readme on where key goes 
#first have everything in pandas df, then think about database
# one module or package w 1 .py 
#adaptors that take in pydantic datatypes and will make into sql
